In [1]:
import pymupdf
import pymupdf4llm
import pandas as pd
import glob
import re
from numpy import float32

In [2]:
all_pdf = glob.glob(r"PDF2\*.pdf")

In [3]:
pdf_document = r"PDF2\Об инциденте с NTP-серверами _ Хабр.pdf"
doc = pymupdf.open(pdf_document)
print("Исходный документ: ", doc.name)
print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
print(doc.metadata)
text = []
for current_page in range(len(doc)):
    page = doc.load_page(current_page)
    page_text = page.get_text("text")
    print("Стр. ", current_page + 1, "\n")
    text.append(page_text)
    print(page_text)

Исходный документ:  PDF2\Об инциденте с NTP-серверами _ Хабр.pdf

Количество страниц: 5

------------------


{'format': 'PDF 1.4', 'title': 'Об инциденте с NTP-серверами / Хабр', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0', 'producer': 'Skia/PDF m136', 'creationDate': "D:20250505174507+00'00'", 'modDate': "D:20250505174507+00'00'", 'trapped': '', 'encryption': None}
Стр.  1 

680.84
Рейтинг
Яндекс
Как мы делаем Яндекс
Подписаться
Недавно на Хабре вышла статья об аномальной нагрузке на публичные NTP‑серверы
в рунете. К сожалению, к этой ситуации привела ошибка в прошивке Яндекс Станций.
Как наши Станции синхронизируют время, что спровоцировало инцидент и как мы
планируем поступить дальше — обо всём этом расскажем в статье. Хочется верить,
что описание ситуации и принятые меры помогут не только нам, но и коллегам
по индустрии избежать подобного в бу

In [4]:
def extract_markdown_text(document) -> str:
    return pymupdf4llm.to_markdown(document)

def extract_text(document) -> str:
    output_text = ""

    doc = pymupdf.open(document)
    for page in doc:
        text = page.get_text("text")
        output_text += text

    return output_text

In [5]:
print(extract_markdown_text(pdf_document))

05.05.2025, 22:45 Об инциденте с NTP-серверами / Хабр

[КАК СТАТЬ АВТОРОМ](https://habr.com/ru/sandbox/start/)
#### **680.84**

Рейтинг
## **Яндекс**

Как мы делаем Яндекс

Подписаться

**[slavashel](https://habr.com/ru/users/slavashel/)**

27 ноя 2024 в 07:54
# Об инциденте с NTP-серверами

**5 мин** **82K**
#### Блог компании Яндекс,  Сетевые технологии*,  Высокая производительность*,  IT-инфраструктура*

Ретроспектива
#### Недавно на Хабре вышла статья об аномальной нагрузке на публичные NTP ‑ серверы в рунете. К сожалению, к этой ситуации привела ошибка в прошивке Яндекс Станций. Как наши Станции синхронизируют время, что спровоцировало инцидент и как мы планируем поступить дальше — обо всём этом расскажем в статье. Хочется верить, что описание ситуации и принятые меры помогут не только нам, но и коллегам по индустрии избежать подобного в будущем.

Динамика числа отправляемых колонками NTP-запросов
## Как Станции обновляют время

https://habr.com/ru/companies/yandex/articles/861538

In [6]:
def parse_markdown(document: str):
    print(document)

In [7]:
def parse_article(document: str):
    text = extract_markdown_text(document)

    lines = text.split("\n")
    result = {}

    for line in lines:
        m = re.match(r"^#{4,} \*\*(?P<rating>[\d.]+)\*\*$", line)
        if m:
            result['rating'] = m.group('rating')
            break

    for i, line in enumerate(lines):
        m = re.match(r"^## \*\*(?P<company>.+)\*\*$", line)
        if m:
            print("company: ", m.group('company'))
            result['company_name'] = m.group('company').strip()
            for desc in lines[i+1:]:
                if desc.strip():
                    result['description'] = desc.strip()
                    break
            break

    m = re.search(r"\*\*\[(?P<author>[^\]]+)\]\([^)]+\)\*\*", text)
    if m:
        result['author'] = m.group('author')

    m = re.search(r"\b(\d{1,2})\s([а-яА-Я]{3,})\s(в)\s(\d{1,2}:\d{2})\b|\b(\d{1,2})\s([а-яА-Я]{3,})\s(\d{4})\s(в)\s(\d{1,2}:\d{2})\b", text)
    if m:
        result['publish_date'] = m.group(0)

    m = re.search(r"^# (?P<title>.+)$", text, flags=re.MULTILINE)
    if m:
        result['title'] = m.group('title').strip()
        result['article_text'] = extract_text(document)
        
    return result

In [8]:
def test():
    file = r"PDF2\Культурный феномен клипа Bad Apple и мой BAD AON _ Хабр.pdf"
    pdf = pymupdf.open(file)
    text = extract_markdown_text(pdf)
    result = parse_article(text)
    print(result)
    print(text)

    file_2 = r"PDF2\Как нас судили за соучастие в преступлении за то, что у нас была лицензия на оказание услуг связи _ Хабр.pdf"
    pdf_2 = pymupdf.open(file_2)
    text = extract_markdown_text(pdf_2)
    result = parse_article(text)
    print(result)
    print(text)

# test()

In [9]:
Author = []
PublishDate = []
Rating = []
ArticleText = []
CompanyName = []
CompanyDescription = []
for pdf in all_pdf:
    result = parse_article(pdf)

    print(result)

    author = result["author"]
    publish_date = result["publish_date"]
    rating = result["rating"]
    article_text = result["article_text"]
    company_name = result["company_name"]
    company_description = result["description"]

    Author.append(author)
    PublishDate.append(publish_date)
    Rating.append(rating)
    ArticleText.append(article_text)
    CompanyName.append(company_name)
    CompanyDescription.append(company_description)

company:  InlyIT
{'rating': '0', 'company_name': 'InlyIT', 'description': 'Для старательного нет ничего невозможного', 'author': 'InlyIT', 'publish_date': '31 мая 2024 в 07:23', 'title': 'Cloudflare положил наш сайт после того, как мы отказались выплатить 120 000 $ в течение 24 часов', 'article_text': '0\nРейтинг\nInlyIT\nДля старательного нет ничего невозможного\nПодписаться\nАвтор оригинала: Robin Dev\nЯ работаю системным оператором в довольно крупном онлайн-казино (мне\nпредставляется, что материал статьи может быть полезен читателям вне зависимости от\nих позиции касательно этичности казино в целом – упоминаю об этом просто для\nконтекста). У нас около четырех миллионов активных пользователей в месяц. С Cloudflare\nмы охотно сотрудничаем с 2018 года по тарифу «Бизнес», который стоит 250 $ в месяц и\nпредоставляет неплохие возможности, включая безлимитный трафик.\nНужно признать, 250 $ — невеликие деньги за те объемы трафика, которые мы прогоняли\nчерез Cloudflare. В основном мы исп

In [10]:
df = pd.DataFrame({
    'Author': pd.Series(dtype=str),
    'PublishDate': pd.Series(dtype=str),
    'Rating': pd.Series(dtype=float32),
    'ArticleText': pd.Series(dtype=str),
    'CompanyName': pd.Series(dtype=str),
    'CompanyDescription': pd.Series(dtype=str)
})

In [11]:
df['Author'] = Author
df['PublishDate'] = PublishDate
df['Rating'] = Rating
df['ArticleText'] = ArticleText
df['CompanyName'] = CompanyName
df['CompanyDescription'] = CompanyDescription

In [12]:
df

,Author,PublishDate,Rating,ArticleText,CompanyName,CompanyDescription
0,InlyIT,31 мая 2024 в 07:23,0,0\nРейтинг\nInlyIT\nДля старательного нет ниче...,InlyIT,Для старательного нет ничего невозможного
1,CodeName33,1 ноя 2024 в 12:01,3257.82,3257.82\nРейтинг\nRUVDS.com\nVDS/VPS-хостинг. ...,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду **HABR15**
2,ru_vds,15 янв в 16:01,3353.13,3353.13\nРейтинг\nRUVDS.com\nVDS/VPS-хостинг. ...,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду **HABR15**
3,km1337,6 мар в 14:00,289.87,289.87\nРейтинг\nH3LLO.CLOUD\nОблачный провайд...,H3LLO.CLOUD,"Облачный провайдер, которого вам захочется обнять"
4,stalkermustang,11 сен 2024 в 07:55,81.85,81.85\nРейтинг\nOpen Data Science\nКрупнейшее ...,Open Data Science,Крупнейшее русскоязычное Data Science сообщество
5,ru_vds,5 фев в 16:01,3387.52,3387.52\nРейтинг\nRUVDS.com\nVDS/VPS-хостинг. ...,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду **HABR15**
6,3_15_10_20,10 дек 2024 в 12:00,680.84,680.84\nРейтинг\nЯндекс\nКак мы делаем Яндекс\...,Яндекс,Как мы делаем Яндекс
7,ntsaplin,14 ноя 2024 в 10:02,3387.52,3387.52\nРейтинг\nRUVDS.com\nVDS/VPS-хостинг. ...,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду **HABR15**
8,shiru8bit,10 окт 2024 в 12:01,3387.52,3387.52\nРейтинг\nRUVDS.com\nVDS/VPS-хостинг. ...,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду **HABR15**
9,mmamayka,15 янв в 11:43,476.08,476.08\nРейтинг\nYADRO\nТут про железо и инжен...,YADRO,Тут про железо и инженерную культуру


## NLP